# Lunar Lander with Cross-Entropy Method

In this notebook we look at the lunar lander environment and solve it with the cross-entropy method.

In [1]:
#!pip3 install 'gymnasium[box2d]'

In [2]:
import gymnasium as gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import matplotlib.pyplot as plt
%matplotlib inline
from collections import deque

torch.manual_seed(1)
np.random.seed(1)

# Neural Network

We define a simple neural network that generates the action scores based on a given state.

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(obs_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, n_actions)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Generate Episodes

We generate a batch of episodes and remember the traversed states, actions and rewards. To select the next action we use the output of the network. For this we first pass the scores through a softmax to get probabilites. In the second step we sampel from this distribution to get the next action to execute.

In [4]:
def generate_batch(env, batch_size, t_max=5000):
    
    activation = nn.Softmax(dim=1)
    batch_actions,batch_states, batch_rewards = [],[],[]
    
    for b in range(batch_size):
        states, actions = [], []
        total_reward = 0
        s, _ = env.reset(seed=0)
        for t in range(t_max):
            
            s_v = torch.FloatTensor([s])
            act_probs_v = activation(net(s_v))
            act_probs = act_probs_v.data.numpy()[0]
            a = np.random.choice(len(act_probs), p=act_probs)

            new_s, r, done, _, _ = env.step(a)

            # record sessions like you did before
            states.append(s)
            actions.append(a)
            total_reward += r

            s = new_s
            if done:
                batch_actions.append(actions)
                batch_states.append(states)
                batch_rewards.append(total_reward)
                break
                
    return batch_states, batch_actions, batch_rewards

# Training

In the training step, we first use the neural network to generate a batch of episodes and then use the state-action pairs to improve the neural network.

In [5]:
batch_size = 100
session_size = 100
percentile = 80
hidden_size = 200
completion_score = 100
learning_rate = 0.01

env = gym.make("LunarLander-v2")
n_states = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(n_states, hidden_size, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=learning_rate)

for i in range(session_size):
    
    # generate new episodes
    states, actions, rewards = generate_batch(env, batch_size, t_max=500)
    
    threshold = np.percentile(rewards, percentile)
    
    # train on the states using actions as targets
    for s_i in range(len(states)):
        if rewards[s_i] < threshold: # skip this iteration if rewards are too low
            continue
        optimizer.zero_grad()
        tensor_states = torch.FloatTensor(states[s_i])
        tensor_actions = torch.LongTensor(actions[s_i])
        action_scores_v = net(tensor_states)
        loss_v = objective(action_scores_v, tensor_actions)
        loss_v.backward()
        optimizer.step()

    #show results
    mean_reward = np.mean(rewards)
    print("%d: loss=%.3f, reward_mean=%.1f" % (
            i, loss_v.item(), mean_reward))
    
    #check if 
    if np.mean(rewards)> completion_score:
        print("Environment has been successfullly completed!")

/var/folders/q7/8hnx45y520d4nlwgykzcbypc0000gn/T/ipykernel_53162/2392018301.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_69nk78ncaj/croot/pytorch_1669252638507/work/torch/csrc/utils/tensor_new.cpp:204.)
  s_v = torch.FloatTensor([s])


0: loss=1.359, reward_mean=-90.8
1: loss=1.366, reward_mean=-107.0
2: loss=1.373, reward_mean=-115.2
3: loss=1.333, reward_mean=-97.9
4: loss=1.401, reward_mean=-98.8
5: loss=1.422, reward_mean=-100.2
6: loss=1.347, reward_mean=-77.4
7: loss=1.344, reward_mean=-63.7
8: loss=1.368, reward_mean=-54.9
9: loss=1.389, reward_mean=-56.0
10: loss=1.376, reward_mean=-50.1
11: loss=1.119, reward_mean=-49.5
12: loss=1.199, reward_mean=-63.7
13: loss=0.956, reward_mean=-42.8
14: loss=0.971, reward_mean=-33.8
15: loss=0.944, reward_mean=-32.1
16: loss=0.871, reward_mean=-76.0
17: loss=0.890, reward_mean=-53.6
18: loss=0.811, reward_mean=-0.6
19: loss=0.895, reward_mean=30.2
20: loss=0.888, reward_mean=29.2
21: loss=0.900, reward_mean=29.5
22: loss=0.808, reward_mean=27.9
23: loss=0.880, reward_mean=41.4
24: loss=0.873, reward_mean=43.5
25: loss=0.850, reward_mean=45.0
26: loss=0.809, reward_mean=41.8
27: loss=0.835, reward_mean=41.8
28: loss=0.811, reward_mean=41.8
29: loss=0.763, reward_mean=48.9

In [6]:
env = gym.make('LunarLander-v2', render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, "video")

state, _ = env.reset()
total_reward = 0.0
done = False

while not done:
        
        state = torch.FloatTensor([state])
        action_scores_v = net(state)
        action = np.argmax(action_scores_v.cpu().data.numpy())

        state, reward, done, _, _ = env.step(action)
        total_reward += reward

env.close()
print(f"Total reward: {total_reward}")

/Users/klausbertsch/miniconda3/envs/rl-course/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/klausbertsch/Documents/GitHub/rl-course-ss25/solutions/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /Users/klausbertsch/Documents/GitHub/rl-course-ss25/solutions/video/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/klausbertsch/Documents/GitHub/rl-course-ss25/solutions/video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/klausbertsch/Documents/GitHub/rl-course-ss25/solutions/video/rl-video-episode-0.mp4
Total reward: 252.14687987987588
